<a href="https://colab.research.google.com/github/CHOPHILSUN/ChoPhilSun-All-Quest/blob/main/ML%20project/CV_Node10_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EB%91%90%EB%B2%88%EC%A7%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 10-3. 프로젝트 2: Object Detection

In [ ]:
# "심화탐구 프로젝트: Object Detection"은 심화 프로젝트로, 과정 수료에 필수는 아닙니다.
# 하지만 여러분들의 실력을 쑥쑥 향상시킬 수 있는 기회이기도 하니, 한번 도전해보시는 것을 권장드립니다!

In [1]:
# 1. Faster RCNN

# 논문: Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks
# https://arxiv.org/pdf/1506.01497.pdf

In [8]:
# RCNN vs Fast R-CNN vs Faster R-CNN
# 2 stage detector(Region Proposal, Classification/Bounding Box Regression)
# content img -> 캡쳐 이미지 삽입 필요
# [RCNN, Fast RCNN, Faster RCNN 비교]
# https://d3s0tskafalll9.cloudfront.net/media/images/NoDeu10_FasterRCNN.max-800x600.png
# https://wikidocs.net/148634

In [ ]:
# Faster RCNN의 구조
# RCNN, Fast RCNN과 Faster RCNN의 가장 큰 차이점은 Selective Search를 Region Proposal Network(RPN) 로 바꾸었다는 것입니다.

# RPN은 RoI가 될 후보군인 Anchor Box를 무수히 만듭니다.
# 그 후 Bounding Box Regression를 통해서 괜찮은 RoI를 딥러닝을 통해 학습하고 Classification을 통해 그것이 Object인지 아닌지를 판단합니다.

# RPN을 통해 얻은 결과에 Object Classification을 통해서 해당 RoI에 있는 Object가 어떤 Class인지 구분하고, Bounding Box Regression을 실행합니다.

#content img <- 캡쳐 이미지 삽입 필요
# [Faster RCNN의 구조]
# https://d3s0tskafalll9.cloudfront.net/media/images/NoDeu10_FasterRCNN_2.max-800x600.png
# https://towardsdatascience.com/faster-r-cnn-for-object-detection-a-technical-summary-474c5b857b46

In [ ]:
# 코드 살펴보기
# 해당 코드는 텐서플로우 공식 홈페이지의 튜토리얼에서 제시된 코드를 활용했습니다.

# https://www.tensorflow.org/hub/tutorials/object_detection?hl=ko

In [28]:
# 먼저 필요한 라이브러리를 불러옵니다.

!pip3 install tensorflow-estimator==2.6.0

In [29]:
# Tensorflow와 Tensoflow Hub에 있는 이미 학습된 모델을 활용하기 위한 모듈
import tensorflow as tf
import tensorflow_hub as hub

# 이미지를 다운로드하는 데에 사용하는 모듈
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# 이미지에 선과 폰트를 넣기 위한 모듈
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# Detection하는 데에 소요된 시간을 점검하기 위한 모듈
import time

ImportError: ignored

In [30]:
# 이미지를 시각화하는 함수를 만들어 둡니다.

def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)

In [31]:
# 원하는 이미지를 다운로드하고 정해진 가로, 세로 길이에 맞게 이미지를 resizing하는 함수를 만들어둡니다.

def download_and_resize_image(url, new_width=256, new_height=256, display=False):

  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)  # 이미지 읽기
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS) # 이미지 조정
  pil_image_rgb = pil_image.convert("RGB") #RBG로 변환
  pil_image_rgb.save(filename, format="JPEG", quality=90) # 이미지 저장
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename

In [32]:
Bounding Box를 만드는 함수를 만듭니다.

def draw_bounding_box_on_image(image,
                               ymin, xmin, ymax, xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):

  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)

# BBox 테두리 그리기
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]

  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height

  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin

SyntaxError: ignored

In [33]:
# 이미지 위에 class 이름과 score를 나타내는 함수도 만듭니다.

# BBox 그리기
def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

# BBox 그리기 적용
  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])  # 박스 좌표값
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

In [34]:
# 이미지를 불러옵니다.

# 원하는 이미지를 불러오고, 가로와 세로를 변경해 보는 실험을 진행해 보세요.


In [35]:
# 새로운 이미지 URL
new_image_url = "https://search.pstatic.net/common/?src=http%3A%2F%2Fimgnews.naver.net%2Fimage%2F047%2F2018%2F08%2F25%2F0002200721_001_20180825134819868.jpg&type=sc960_832"

# 이미지 다운로드 및 저장
new_downloaded_image_path = download_and_resize_image(new_image_url, 1280, 856, True)

# 테스트
run_detector(detector, new_downloaded_image_path)


NameError: ignored

In [36]:
# tfhub에서 사전 학습된(Pre-trained) InceptionResnet 모델을 사용한 Faster RCNN 모델을 불러옵니다.

module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

detector = hub.load(module_handle).signatures['default'] # detector에 사용할 모듈 저장

NameError: ignored

In [37]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [38]:
# Detector를 실행하는 함수를 만듭니다.

def run_detector(detector, path):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

# 소요시간 측정
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

# 이미지 추론 개수와 추론 시간 출력
  print("Found %d objects." % len(result["detection_scores"]))
  print("Inference time: ", end_time-start_time)

# 이미지 내 박스로 entity, scores를 추가하여 출력
  image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"])

  display_image(image_with_boxes)

In [39]:
# detector를 실행시켜 봅시다.

run_detector(detector, downloaded_image_path)

NameError: ignored

In [40]:
# 2-1. 다른 이미지 테스트해보기
# 원하는 이미지를 불러와 결과를 확인해 봅시다.

# 이미지 불러오기

image_path = "/path/to/your/image.jpg"  # 경로를 실제 이미지 파일의 경로로 변경 필요
run_detector(detector, image_path)

NameError: ignored

In [41]:
# 테스트

# 예를 들어, 2번째 모델의 성능을 평가하는 코드를 작성해보겠습니다.
# 이 코드는 실제로 사용하고자 하는 모델에 따라 다를 수 있습니다.

# 테스트 데이터셋 로드 및 전처리 (예시)
test_data_gen = ...  # 테스트 데이터셋 생성

# 모델 평가
test_loss, test_accuracy = model.evaluate(test_data_gen)

# 결과 출력
print(f"테스트 손실: {test_loss}")
print(f"테스트 정확도: {test_accuracy}")

NameError: ignored

In [42]:
# 문제 2-2. 다양한 pre-trained model 테스트해보기
# Faster RCNN 외에도 Object Detection을 할 수 있는 다양한 사전 학습 모델이 있습니다.
# tfhub을 탐색하여 사용할 수 있는 1개 이상의 모델을 불러와 결과를 테스트해보고, 그 결과를 비교해 보세요.

In [43]:
# 문제 2-2. 모델 불러오기

# 테스트
# 테스트하고 싶은 이미지의 URL을 넣어주세요.
test_image_url = "https://example.com/your-test-image.jpg"

# 이미지 다운로드 및 테스트
downloaded_test_image_path = download_and_resize_image(test_image_url, 1280, 856, True)
run_detector(detector, downloaded_test_image_path)

NameError: ignored

In [44]:
# 문제 2-2. 테스트

# 테스트하고 싶은 이미지의 URL을 넣어주세요.
test_image_url_2 = "https://example.com/your-test-image-2.jpg"

# 이미지 다운로드 및 테스트
downloaded_test_image_path_2 = download_and_resize_image(test_image_url_2, 1280, 856, True)
run_detector(detector, downloaded_test_image_path_2)

NameError: ignored

In [45]:
# 문제 2-2. 결과 비교
print("1번째 모델인 Inception ResNet을 사용한 Faster RCNN은 조금 느리지만 높은 정확성을 보였습니다.")
print("2번째 모델인 [YOUR MODEL NAME]은 ...")  # 여기에 2번째 모델의 성능에 대한 특징을 추가가 필요함.

1번째 모델인 Inception ResNet을 사용한 Faster RCNN은 조금 느리지만 높은 정확성을 보였습니다.
2번째 모델인 [YOUR MODEL NAME]은 ...


In [ ]:
'''
루브릭
아래의 기준을 바탕으로 프로젝트를 평가합니다.
평가문항	상세기준
1. VGG16 모델을 구현할 수 있는가?	이미지로 제시된 VGG16 모델을 코드로 구현하였다.
2. 다양한 방법을 사용하여 성능을 향상시켰는가?	다양한 방법을 사용하여 accuracy 53% 이상을 달성하였다.
3. 다양한 이미지와 모델을 사용하여 Object Detection을 수행하였는가?	제시된 이미지 외의 다른 이미지에 Object Detection을 수행하였고, 1가지 이상의 사전 학습된 모델을 사용하여 결과를 비교하였다.
'''


In [ ]:
# 조필선 회고록

'''
이번 프로젝트에서는 VGG16 모델을 온전히 구현하는 데 어려움을 겪었습니다.
성능이 향상되도록 시도를 하였지만 예상치 못하게 코드실행시 마다 에러가 발생하므로 실패하였습니다.
Object Detection을 위한 이미지와 모델 실험도 중간에 문제가 생겨 목표를 달성하지 못했습니다.
더 나은 결과를 위해 에러를 해결하도록 노력해야 할 것 같습니다.
어쨌든 이번에 프로젝트를 완료하지 못해서 죄송합니다.

<내가 배운 점들>

1. 이번 프로젝트에서 VGG16 모델의 구현 및 성능 향상 시도를 통해서 딥러닝 모데을 이해하는데 많은 도움이 되었습니다.
   모델 구현시에는 각 레이어의 구성과 파라미터 설정에 대한 이해가 필요하다는 것을 알 수가 있었고 코드를 작성하면서도 구조적인 부분에 대해서
   조금 더 깊게 알게 된것 같습니다.

2. 성능 향상을 위한 다양한 방법을 적용하려고 해보았으나 예상치 못한 문제들(LMS 엔진커널 문제,속도 문제 등)로 제대로 된 결과를 내기가 어려웠습니다.
   이러한 어려움이 현업에서 일을 하게 됢때 모델을 구현하고 튜닝할 때 부딪힐 수 있는 현실적인 상황이라는 점이 마음에 와 닿았습니다.
   따라서 에러 해결과 실험 설계 능력을 향상시키기 위해서는 나 스스로 많은 학습의 노력이 필요하다는 것도 깨달았습니다.

3. Object Detection을 위한 다양한 이미지와 사전 학습된 모델을 사용한 경험이 중요하다는 것을 알게 되었습니다.
'''